In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import anndata as ad

In [3]:
ref = sc.read_h5ad("/path/to/00.dataset/level3_cp_ctrl.h5ad")

In [4]:
import numpy as np
import scanpy as sc

np.random.seed(123)


target_cells = ['A549']


selected_indices = []

for cell_line in target_cells:
    cell_mask = ref.obs['cell'] == cell_line
    
    cell_indices = ref.obs_names[cell_mask].tolist()
    
    if len(cell_indices) < 3:
        print(f"Warning: {cell_line} only {len(cell_indices)} records")
        continue
    
    selected = np.random.choice(cell_indices, size=3, replace=False)
    selected_indices.extend(selected)

subset_adata = ref[selected_indices].copy()

print(subset_adata.obs['cell'].value_counts())

最终子集维度: (3, 978)

各细胞系样本数量:
cell
A549    3
Name: count, dtype: int64


In [6]:
drug_df = pd.read_csv("/path/to/Targetmol_L6000.csv", encoding='latin1')
drug_df

,ID,Name,CAS,SMILES
0,TWS2045,Bruceine D,21499-66-1,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...
1,TWS1563,Incensole,22419-74-5,CC(C)[C@]12CC[C@@](C)(O1)[C@H](O)CC\C(C)=C\CC\...
2,TWS1293,Angelic anhydride,94487-74-8,C\C=C(\C)C(=O)OC(=O)C(\C)=C/C
3,TWS1229,Aristolone,25274-27-5,C[C@@H]1CCCC2=CC(=O)[C@@H]3[C@@H](C3(C)C)[C@@]12C
4,TWS0787,Oroxin A,31567-75-6,OC[C@H]1O[C@@H](Oc2cc3oc(cc(=O)c3c(O)c2O)-c2cc...
...,...,...,...,...
1856,T0021,Vanatone,70-70-2,CCC(=O)c1ccc(O)cc1
1857,T0019,Betaine hydrochloride,590-46-5,[Cl-].C[N+](C)(C)CC(O)=O
1858,T0014,cis-Anethol,104-46-1,COc1ccc(C=CC)cc1
1859,T0012,Cinchonine,118-10-5,O[C@H]([C@H]1C[C@@H]2CC[N@]1C[C@@H]2C=C)c1ccnc...


In [7]:
subset_adata

AnnData object with n_obs × n_vars = 3 × 978
    obs: 'smiles', 'cell'
    layers: 'unpert_expr'

In [8]:
unpert_expr = subset_adata.layers['unpert_expr']
obs_subset = subset_adata.obs.copy()
n_cells = unpert_expr.shape[0]
n_genes = unpert_expr.shape[1]

expr_list = []
obs_list = []


for idx, row in drug_df.iterrows():
    expr_repeated = unpert_expr.copy()
    expr_list.append(expr_repeated)
    
    obs_repeated = obs_subset.copy()
    obs_repeated = obs_repeated.reset_index(drop=True)
    obs_repeated['IDNUMBER'] = row['ID']
    obs_repeated['Cas'] = row['CAS']
    obs_repeated['drug_SMILES'] = row['SMILES']
    obs_list.append(obs_repeated)

X_new = np.vstack(expr_list)
obs_new = pd.concat(obs_list, ignore_index=True)
var_new = subset_adata.var.copy()

adata_new = ad.AnnData(X=X_new, obs=obs_new, var=var_new)
adata_new.layers['unpert_expr'] = X_new.copy()
adata_new.X = X_new.copy()


/slurm/home/yrd/liaolab/lvtianhang/anaconda3/envs/transformer/lib/python3.10/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [9]:
adata_new.obs

,smiles,cell,IDNUMBER,Cas,drug_SMILES
0,COc1cc2CC[N+](C)(C)[C@@H]3Cc4ccc(O)c(Oc5cc6[C@...,A549,TWS2045,21499-66-1,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...
1,Cc1ccc(NCn2c(=O)oc3ccccc23)cc1,A549,TWS2045,21499-66-1,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...
2,CCN(Cc1ccccc1)C(=O)Cn1c2nc(ncc2n(C)c1=O)-c1ccccc1,A549,TWS2045,21499-66-1,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...
3,COc1cc2CC[N+](C)(C)[C@@H]3Cc4ccc(O)c(Oc5cc6[C@...,A549,TWS1563,22419-74-5,CC(C)[C@]12CC[C@@](C)(O1)[C@H](O)CC\C(C)=C\CC\...
4,Cc1ccc(NCn2c(=O)oc3ccccc23)cc1,A549,TWS1563,22419-74-5,CC(C)[C@]12CC[C@@](C)(O1)[C@H](O)CC\C(C)=C\CC\...
...,...,...,...,...,...
5578,Cc1ccc(NCn2c(=O)oc3ccccc23)cc1,A549,T0012,118-10-5,O[C@H]([C@H]1C[C@@H]2CC[N@]1C[C@@H]2C=C)c1ccnc...
5579,CCN(Cc1ccccc1)C(=O)Cn1c2nc(ncc2n(C)c1=O)-c1ccccc1,A549,T0012,118-10-5,O[C@H]([C@H]1C[C@@H]2CC[N@]1C[C@@H]2C=C)c1ccnc...
5580,COc1cc2CC[N+](C)(C)[C@@H]3Cc4ccc(O)c(Oc5cc6[C@...,A549,T0011,60-12-8,OCCC1=CC=CC=C1
5581,Cc1ccc(NCn2c(=O)oc3ccccc23)cc1,A549,T0011,60-12-8,OCCC1=CC=CC=C1


In [10]:
drug_df

,ID,Name,CAS,SMILES
0,TWS2045,Bruceine D,21499-66-1,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...
1,TWS1563,Incensole,22419-74-5,CC(C)[C@]12CC[C@@](C)(O1)[C@H](O)CC\C(C)=C\CC\...
2,TWS1293,Angelic anhydride,94487-74-8,C\C=C(\C)C(=O)OC(=O)C(\C)=C/C
3,TWS1229,Aristolone,25274-27-5,C[C@@H]1CCCC2=CC(=O)[C@@H]3[C@@H](C3(C)C)[C@@]12C
4,TWS0787,Oroxin A,31567-75-6,OC[C@H]1O[C@@H](Oc2cc3oc(cc(=O)c3c(O)c2O)-c2cc...
...,...,...,...,...
1856,T0021,Vanatone,70-70-2,CCC(=O)c1ccc(O)cc1
1857,T0019,Betaine hydrochloride,590-46-5,[Cl-].C[N+](C)(C)CC(O)=O
1858,T0014,cis-Anethol,104-46-1,COc1ccc(C=CC)cc1
1859,T0012,Cinchonine,118-10-5,O[C@H]([C@H]1C[C@@H]2CC[N@]1C[C@@H]2C=C)c1ccnc...


In [15]:
import pandas as pd
import numpy as np
import anndata as ad

In [16]:
unpert_expr = []
smiles_ls = []
cas_ls = []
id_ls = []
cell_ls = []

In [17]:
for idx,row in drug_df.iterrows():
    smiles = row['SMILES']
    smiles_ls += [smiles] * len(subset_adata)
    cas_ls += [row['CAS']] * len(subset_adata)
    id_ls += [row['ID']] * len(subset_adata)
    cell_ls += subset_adata.obs['cell'].tolist()
    
    unpert_expr.append(subset_adata.layers['unpert_expr'])

In [18]:
unpert_expr_np = np.vstack(unpert_expr)

In [19]:
adata_new = ad.AnnData(X=unpert_expr_np)

In [20]:
adata_new.layers['unpert_expr'] = unpert_expr_np
adata_new

AnnData object with n_obs × n_vars = 5583 × 978
    layers: 'unpert_expr'

In [21]:
adata_new.obs['smiles'] = smiles_ls
adata_new.obs['Cas'] = cas_ls
adata_new.obs['IDNUMBER'] = id_ls
adata_new.obs['cell'] = cell_ls
adata_new.obs

,smiles,Cas,IDNUMBER,cell
0,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...,21499-66-1,TWS2045,A549
1,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...,21499-66-1,TWS2045,A549
2,CC1=CC(=O)[C@@H](O)[C@@]2(C)[C@H]1C[C@H]1OC(=O...,21499-66-1,TWS2045,A549
3,CC(C)[C@]12CC[C@@](C)(O1)[C@H](O)CC\C(C)=C\CC\...,22419-74-5,TWS1563,A549
4,CC(C)[C@]12CC[C@@](C)(O1)[C@H](O)CC\C(C)=C\CC\...,22419-74-5,TWS1563,A549
...,...,...,...,...
5578,O[C@H]([C@H]1C[C@@H]2CC[N@]1C[C@@H]2C=C)c1ccnc...,118-10-5,T0012,A549
5579,O[C@H]([C@H]1C[C@@H]2CC[N@]1C[C@@H]2C=C)c1ccnc...,118-10-5,T0012,A549
5580,OCCC1=CC=CC=C1,60-12-8,T0011,A549
5581,OCCC1=CC=CC=C1,60-12-8,T0011,A549


In [22]:
adata_new

AnnData object with n_obs × n_vars = 5583 × 978
    obs: 'smiles', 'Cas', 'IDNUMBER', 'cell'
    layers: 'unpert_expr'

In [23]:
adata_new.write_h5ad("NSCLC_target_mol.step1.h5ad")

# prediction

In [24]:
import sys
sys.path.append("/path/to/wave")
from model import WAVE
import torch
import numpy as np
import scanpy as sc

In [25]:
model = WAVE()

checkpoint_path = "/path/to/best_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device("cpu")))

model.eval()

WAVE(
  (gene_vae): GeneVAE(
    (encoder): Sequential(
      (0): Linear(in_features=978, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): ReLU()
    )
    (fc_mu): Linear(in_features=256, out_features=128, bias=True)
    (fc_logvar): Linear(in_features=256, out_features=128, bias=True)
    (decoder): Sequential(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=978, bias=True)
    )
  )
  (drug_feature): DrugNN(
    (drug_fc): Sequential(
      (0): Linear(in_features=2048, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=128, bias=True)
      (3): ReLU()
    )
  )
  (fusion): GeneDrugFusion(
    (fusion_layer): Sequential(
      (0): Linear(in_features=384, out_features=512, bias=True)
      (1): ReLU()
   

In [26]:
adata = sc.read_h5ad("NSCLC_target_mol.step1.h5ad")

In [27]:
adata.layers['unpert_expr'] = adata.X

In [28]:
gene_expr = adata.layers["unpert_expr"]  # shape: [n, 978]
smiles_list = adata.obs["smiles"].tolist()

In [29]:
def smiles_to_fp(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(nBits, dtype=np.float32)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
    arr = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr.astype(np.float32)

In [30]:
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem

In [31]:
drug_fp_array = np.array([smiles_to_fp(s) for s in smiles_list], dtype=np.float32)

# 5. 转换为 PyTorch tensor
gene_expr_tensor = torch.tensor(np.array(gene_expr), dtype=torch.float32)
drug_fp_tensor = torch.tensor(drug_fp_array, dtype=torch.float32)

In [32]:
drug_fp_tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [33]:
with torch.no_grad():
    predicted_expr,_,_,_ = model(gene_expr_tensor, drug_fp_tensor)

In [34]:
predicted_expr_np = predicted_expr.numpy()

In [35]:
adata.layers["pred"] = predicted_expr_np

In [36]:
adata.layers["pred"][:5]

array([[ 9.823633 ,  5.0370574,  9.72896  , ..., 12.027216 , 10.776029 ,
         7.926386 ],
       [10.257696 ,  5.0484357,  9.616388 , ..., 12.186471 , 10.951464 ,
         8.339371 ],
       [ 9.539898 ,  5.6338716,  8.66603  , ..., 10.493118 , 10.104688 ,
         7.6967106],
       [ 9.720716 ,  4.830626 ,  9.427818 , ..., 11.958947 , 10.716544 ,
         7.864292 ],
       [10.139107 ,  5.222512 ,  9.063983 , ..., 11.746427 , 10.576273 ,
         8.413297 ]], dtype=float32)

In [37]:
adata.layers["unpert_expr"][:5]

array([[ 9.676825 ,  4.2863503, 10.3295   , ..., 12.12225  , 10.041349 ,
         8.1269245],
       [10.41385  ,  4.1682   ,  8.4472   , ..., 11.34045  , 11.5521   ,
         8.0179   ],
       [ 9.4435425,  6.009461 ,  8.44984  , ..., 10.036426 , 10.839643 ,
         7.5686936],
       [ 9.676825 ,  4.2863503, 10.3295   , ..., 12.12225  , 10.041349 ,
         8.1269245],
       [10.41385  ,  4.1682   ,  8.4472   , ..., 11.34045  , 11.5521   ,
         8.0179   ]], dtype=float32)

In [42]:
adata.var_names = subset_adata.var_names

In [43]:
adata.write_h5ad("LUAD_target_mol_pred.step2.h5ad")